In [1]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [2]:
from summary_eval.data import summary_df, prompts_df
from summary_eval.settings import TRAIN_SIZE
from summary_eval.testing import cross_validate

2024-02-29 18:26:14,514 - INFO - Read 7165 summaries from C:\Users\theaw\summary-eval\data\summaries_train.csv
2024-02-29 18:26:14,529 - INFO - Read 4 prompts from C:\Users\theaw\summary-eval\data\prompts_train.csv


## View data

In [3]:
summary_df.head()

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


In [4]:
prompts_df.head()

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


## Create basic feature and split into train, test, X, y

In [5]:
summary_df["summary_length"] = summary_df["text"].str.len()
summary_df["summary_length"]

0        346
1        244
2       1370
3        157
4       1225
        ... 
7160     357
7161     255
7162     272
7163     294
7164     530
Name: summary_length, Length: 7165, dtype: int64

In [6]:
features = ["summary_length"]
X = summary_df[features]

targets = ["content", "wording"]
y = summary_df[targets]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_SIZE, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

(5732, 1433, 5732, 1433)

## Train multi-output model

In [8]:
model = MultiOutputRegressor(GradientBoostingRegressor(random_state=0))
results_df = cross_validate(model, X_train, y_train)
results_df

Metric      rmse                                 mae            \
Target   content   wording mean_columnwise   content   wording   
mean    0.529941  0.818091        0.674016  0.397817  0.634637   
stdev    0.01936  0.017414        0.018387  0.009998  0.013478   

Metric                        r2                            
Target mean_columnwise   content   wording mean_columnwise  
mean          0.516227  0.739797  0.381257        0.560527  
stdev         0.011738  0.030221  0.039401        0.034811

### The metric the competition cares about:

In [9]:
results_df["rmse"]["mean_columnwise"]

mean     0.674016
stdev    0.018387
Name: mean_columnwise, dtype: object